In [10]:
import numpy as np
import cv2
import pandas as pd
import random

In [11]:
df_pred = pd.read_csv('./sorted_predictions_pred_2756-3552.csv')
df = pd.read_csv('./old/match.csv')
df = df.sort_values('time')
f_ids = df['frame_id_forward'].values[2756:3552]
carv = df_pred['prediction'].values
num_y = len(carv)

In [11]:
df = pd.read_csv('./match.csv')
df = df.sort_values('time')
f_ids = df['frame_id_forward'].values
velocity = df['speed'].values
omega = df['angular_velocity'].values
carv = np.zeros(len(velocity))
for i in range(len(velocity)):
    carv[i] = omega[i]/velocity[i] if velocity[i] >= 0.27 else 0
num_y = len(carv)

In [12]:
def get2Bezier(P0, P1, P2, t):
    a0 = (1-t)**2
    a1 = 2 * (1-t) * t
    a2 = t**2
    x = a0*P0[0] + a1*P1[0] + a2*P2[0]
    y = a0*P0[1] + a1*P1[1] + a2*P2[1]
    return np.array([int(x), int(y)])

def routed_result(Ids, carv, inpath, outpath):
    '''
    Input:
        Ids: a dataframe or list of images' Ids in type of string or integer 
        carv: the carvinature list or array computed by speed and angle_speed in order
        inpath: the file path for forward camera images
        outpath: the file path to save the generated images
    Output:
        Save the predicted direction visualization on the forward camera image to outpath
    '''
    P0 = [320,402]
    P1 = [320,241]
    for r in range(len(Ids)):
        k = carv[r]
        f_id = Ids[r]
        P2 = [320-k*2000,241]
        path = inpath+"frame_vc1_%d.bmp"%(int(f_id))
        img = cv2.imread(path)
        pts = np.zeros((11,2))
        for t in range(11):
            pts[t,:] = get2Bezier(P0,P1,P2,t/10)
        pts = pts.reshape((11,1,2))
        pts = pts.astype(np.int32)
        cv2.polylines(img,[pts],False,(0, 0, 255),5) 
        cv2.imwrite(outpath+"frame_vc1_%d.bmp"%(int(f_id)), img)
        if r%100 == 0:
            print('{r}/{num} images processed'.format(r=r, num=len(Ids)))

In [13]:
for i in range(num_y):
    k = carv[i]
    f_id = f_ids[i]
    P0 = [320,402]
    P1 = [320,241]
    P2 = [320-k*1300,241]
    path = "./old/forward_compressed/frame_vc1_%d.bmp"%(int(f_id))
    img = cv2.imread(path)
    pts = np.zeros((11,2))
    for t in range(11):
        pts[t,:] = get2Bezier(P0,P1,P2,t/10)
    # pts = np.array([[320,402],[320,241],[640,241]],np.int32)
    pts = pts.reshape((11,1,2))
    pts = pts.astype(np.int32)
    cv2.polylines(img,[pts],False,(0, 0, 255),5) 

    #为了演示，建窗口显示出来
#     cv2.namedWindow('image',cv2.WINDOW_NORMAL)
#     cv2.resizeWindow('image',1000,1000)#定义frame的大小
#     cv2.imshow('image',img)
    if i%100 == 0:
        print(f'{i}/{num_y} images processed')
    cv2.imwrite("./old_test_result_pred/frame_vc1_%d.bmp"%(int(f_id)), img)

0/768 images processed
100/768 images processed
200/768 images processed
300/768 images processed
400/768 images processed
500/768 images processed
600/768 images processed
700/768 images processed
